In [ ]:
import cv2
import numpy as np
import os

### MATRIZ BCH

Esta matríz se ha obtenido del artículo científico: `Brightness Calculation in Digital Image Processing` y nos ayudará a calcular el brillo de una imagen.

In [ ]:
MATRIZ_toDEF = np.array([[0.2053, 0.7125, 0.4670],
                        [1.8537, -1.2797, -0.4429],
                        [-0.3655, 1.0120, -0.6014]])

print(MATRIZ_toDEF) 

In [ ]:
MATRIZ_toXYZ = np.array([[0.6712, 0.4955, 0.1540],
                        [0.7061, 0.0248, 0.5223],
                        [0.7689, -0.2556, -0.8645]])

### 🧮 Cálculo del Brillo utilizando la Métrica de Cohen  

La función `obtener_brillo` calcula el brillo de una imagen a partir de sus componentes en el espacio de color XYZ, utilizando la métrica de Cohen basada en el modelo BCH (Brightness, Chroma, Hue).  

**¿Cómo funciona?**  
- Transformación BCH:  
   - Se toma la imagen en sus componentes **X, Y, Z** (R, G, B) y se aplica la matriz auxiliar para obtener nuevas representaciones **D, E, F**.    

- Cálculo brillo:
   - A partir de los valores **D, E, F**, se calcula el brillo **B** como la norma euclidiana en este espacio:  
     
     ```math
     B = \sqrt{D^2 + E^2 + F^2}
     ```

In [ ]:
def calc_metricas_cohen(X, Y, Z):
    alto, ancho = X.shape
    # Stack y reshape en una matriz (N, 3)
    pixels_xyz = np.stack([X, Y, Z], axis=-1).reshape(-1, 3)
    # Aplicar la matriz DEF
    cohen = pixels_xyz @ MATRIZ_toDEF.T  # Resultado (N, 3)
    return cohen.reshape(alto, ancho, 3)

def calc_bch_to_xyz(B, C, H):
    alto, ancho = B.shape
    # Stack y reshape en una matriz (N, 3)
    pixels_bch = np.stack([B, C, H], axis=-1).reshape(-1, 3)
    # Aplicar la matriz XYZ
    xyz = pixels_bch @ MATRIZ_toXYZ.T  # Resultado (N, 3)
    return xyz.reshape(alto, ancho, 3)

def obtener_brillo_y_valores_def(X, Y, Z):
    cohen = calc_metricas_cohen(X, Y, Z)
    D, E, F = cohen[:, :, 0], cohen[:, :, 1], cohen[:, :, 2]
    B = np.sqrt(D**2 + E**2 + F**2)
    return B.mean(), D, E, F

### 🧮 Obtener brillo y canal DEF de la imagen

La función `obtener_def_imagen` aplica las funciones creadas en la celda anterior y devuelve el valor del canal DEF.

**¿Cómo funciona?**  
- Se transforma el formato de la imagen para que los valores estén entre 0 y 256 B
- Transforma la imagen en X Y Z a partir de rgb y se guarda cada canal
- Le pasa cada componente a la función `obtener_brillo_y_valores_def` y guarda valor D E F
- Devuelve los canales D E F por separado

In [ ]:
def obtener_def_imagen(imagen_rgb):
    imagen_float = imagen_rgb.astype(np.float32) / 255.0

    X, Y, Z = cv2.split(cv2.cvtColor(imagen_float, cv2.COLOR_RGB2XYZ))
    
    _, D, E, F = obtener_brillo_y_valores_def(X, Y, Z)
    return D, E, F

### 🧮 Cargar imagen RGB   

La función `cargar_imagen_rgb` es una función auxiliar que carga una imagen en rgb .  


In [ ]:
def cargar_imagen_rgb(ruta):
    img = cv2.imread(ruta)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen: {ruta}")
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

### 🗂️ Procesamiento masivo de imágenes calculo del brillo

Esta función `procesar_imagenes_carpeta_bch` aplica la métrica de Cohen para calcular el brillo a todas las imágenes `.png` dentro de una estructura de carpetas.

**Qué hace:**
- Recorre subcarpetas dentro de una carpeta principal.
- Carga la imagen en RGB.
- Aplica la métrica de cohen para obtenter el brillo .
- Guarda el valor del brillo obtenido en un archivo `.txt`.

In [ ]:
def procesar_imagenes_carpeta_bch(
        carpeta_entrada="images",
        carpeta_salida="images_procesadas"
):
    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)
        
        # Verificamos si es una carpeta
        if not os.path.isdir(ruta_subcarpeta):
            continue
        
        # Crear subcarpeta de salida correspondiente
        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta)

        # Añadimos la subcarpeta específica para la luminosidad
        carpeta_salida_sub = os.path.join(carpeta_salida_sub, "luminosidad")
        os.makedirs(carpeta_salida_sub, exist_ok=True)
        
        # Creamos un archivo TXT para guardar los brillos
        ruta_txt = os.path.join(carpeta_salida_sub, "info_brillos_bch.txt")

        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write("Nivel de Brillo (método BCH)\n")
            archivo_txt.write(f"Carpeta de imágenes: {ruta_subcarpeta}\n\n")
            archivo_txt.write("Valores mayores corresponden a mayor brillo.\n\n")
            
            # Recorremos los archivos dentro de la subcarpeta
            for filename in sorted(os.listdir(ruta_subcarpeta)):
                if filename.lower().endswith(("sharp.png", "bright.png", "dark.png")):
                    ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)
                    
                    # Cargar imagen en rgb
                    img_rgb = cargar_imagen_rgb(ruta_imagen_entrada)
                    
                    # Calcular brillo
                    imagen_float = img_rgb.astype(np.float32) / 255.0
                    X, Y, Z = cv2.split(cv2.cvtColor(imagen_float, cv2.COLOR_RGB2XYZ))
                    brillo = obtener_brillo_y_valores_def(X, Y, Z) [0]

                    
                    # Guardar resultado en el TXT
                    archivo_txt.write(f"{filename} -> brillo: {brillo:.4f}\n")

    print("Procesamiento completado con método BCH.")

In [ ]:
procesar_imagenes_carpeta_bch(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas"
)

### 🧮 Modificar brillo de una imagen   
**¿Cómo funciona?**  
- Se determina un coeficiente ventana al llamar a la función que establece la cantidad de píxeles que se utilizarán para calcular la media del brillo.
- Se determina un coeficiente k al llamar a la función que establece la intensidad del contraste.
- Se calcula el brillo de la imagen utilizando la matriz BCH.
- Se obtiene el brillo promedio de la imagen con la funcion `calcular_b_promedio`.
- Se aplica la formula del artículo científico para modificar el brillo.
- Se modifican los canales de la imagen con los nuevos valores.
- Devuelve la imagen con el nuevo contraste.

In [ ]:
def modificar_brillo(imagen_rgb, ev):
    """
    Implementa la mejora de brillo preservando las coordenadas cromáticas
    según la fórmula (5) del paper
    """
    # Obtener componentes D, E, F
    D, E, F = obtener_def_imagen(imagen_rgb)
    
    # Calcular brillo como la norma euclidiana de D, E, F
    epsilon = 1e-10
    brillo = np.sqrt(D**2 + E**2 + F**2)
    
    # Calcular C y H según las fórmulas del modelo BCH
    # C = arccos(D/B)
    C = np.arccos(np.clip(D / (brillo + epsilon), -1.0, 1.0))
    
    # H = arctan2(F, E) - Usando arctan2 para obtener el ángulo completo en el rango [-π, π]
    sin_C = np.sin(C)
    # Evitar división por cero
    mask = sin_C > epsilon
    H = np.zeros_like(C)
    
    # E = B*sin(C)*cos(H),  F = B*sin(C)*sin(H)
    # tan(H) = F/E  →  H = arctan2(F, E)
    # Usar arctan2 para determinar el ángulo correctamente en todos los cuadrantes
    H[mask] = np.arctan2(F[mask], E[mask])
        
    # Aplicar la fórmula de modificación de brillo
    nuevo_B = np.power(2, ev) * brillo
    
    # Limitar valores al rango válido
    nuevo_B = np.clip(nuevo_B, 0, 255)
    
    # Reconstruir D, E, F usando las fórmulas del modelo BCH
    # D = B * cos(C)
    nuevo_D = nuevo_B * np.cos(C)
    # E = B * sin(C) * cos(H)
    nuevo_E = nuevo_B * np.sin(C) * np.cos(H)
    # F = B * sin(C) * sin(H)
    nuevo_F = nuevo_B * np.sin(C) * np.sin(H)
    
    # Convertir de DEF a XYZ
    nuevoXYZ = calc_bch_to_xyz(nuevo_D, nuevo_E, nuevo_F)
    
    # Asegurar que nuevoXYZ esté en el formato correcto
    nuevoXYZ = nuevoXYZ.astype(np.float32)
    
    # Convertir de vuelta a RGB
    resultado = cv2.cvtColor(nuevoXYZ, cv2.COLOR_XYZ2RGB) * 255
    
    # Retornar como uint8
    return np.clip(resultado, 0, 255).astype(np.uint8)
